`pip install -U msgpack matplotlib pyvisa`

In [ ]:
from typing import Literal
from time import sleep
import numpy as np, csv
from experiment import Experiment
from pyvisa import ResourceManager

def read(typ: Literal['HIGH', 'MEAN', 'LOW']):
    # 读取脉冲幅度的测量值
    scope.write('MEASUrement:IMMed:SOUrce CH1')  # 设置测量源为通道 1
    scope.write('MEASUrement:IMMed:TYPe ' + typ)  # 设置测量类型为脉冲幅度
    return float(scope.query('MEASUrement:IMMed:VALUE?'))  # 读取脉冲幅度

def write(ch: int, amp: float):
    console = Experiment(init_gpa=False)
    console.add_flodict({
        f'ocra40_v{ch}': (np.array([10, 10000, 20000]), np.array([0, amp, 0])),
    })
    msg = console.run()
    console.__del__()
    return msg

def sweep(ch: int):
    results = []
    def expt(amp: float):
        write(ch, amp)
        sleep(1)
        high = read('HIGH')
        results.append([amp, high])
        if np.abs(high) > 26:
            results.append([0, read('LOW')])
            return True
        return False
    
    for amp in np.linspace(0, 0.3, 31):
        if expt(amp):
            break
    for amp in np.linspace(0, -0.3, 31):
        if expt(amp):
            break
    with open(f'./data/{ch}.txt', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(results)
    return np.array(results)

try:
    scope.close()
except NameError:
    pass
rm = ResourceManager()
scope = rm.open_resource(rm.list_resources()[0])
scope.query('*IDN?')

In [ ]:
scope.close()

In [ ]:
console = Experiment(init_gpa=True)
console.add_flodict({
    f'ocra40_v{ch}': (np.array([0, 100]), np.array([0, 0])) for ch in range(40)
})
console.run()
console.__del__()

In [ ]:
write(0, 0.1)

In [ ]:
sweep(0)